In [1]:
import sys
import os
import warnings

sys.path.append(
    os.path.abspath(os.path.join(os.path.dirname(os.path.realpath("__file__")), ".."))
)

warnings.simplefilter(action="ignore", category=FutureWarning)

In [2]:
import pandas as pd
from datetime import date
import time

import data_loader.yahoo_finance as yahoo_finance
import utils.downloading_utils as downloading_utils
import utils.azure_utils as azure_utils
import utils.mapping as mapping

import handyman.holdings as holdings
import handyman.prices as prices
import handyman.company_info as company_info
from utils.yfinance_utils import (
    pull_prices,
    pull_financials,
    create_client,
    pull_officers,
    pull_info,
)
from utils.database_utils import find_missing_tickers

In [3]:
engine = azure_utils.get_azure_engine(configs_path="../configs/configs.json")

# Download Holdings Files

In [4]:
etf_urls = mapping.etf_urls
data_path = os.path.join(
    os.path.abspath(os.path.join(os.path.dirname(os.path.realpath("__file__")), "..")),
    "Data",
)

all_indices = pd.DataFrame()
for fund_name, url in etf_urls.items():
    print(fund_name)
    df_temp = downloading_utils.download_holdings(
        fund_name, url, download_folder=data_path
    )
    all_indices = pd.concat([all_indices, df_temp])

S&P 500
Russell 1000


In [4]:
azure_utils.write_sql_table(
    engine=engine, table_name="HOLDINGS", df=all_indices, overwrite=False
)

NameError: name 'all_indices' is not defined

In [4]:
all_indices = holdings.get_index_holdings(
    # indices=['S&P 500'],
    # tickers=["AAPL"],
    start_date="2025-12-29",
)

In [ ]:
# azure_utils.delete_sql_rows(
#     engine=engine,
#     table_name="HOLDINGS",
#     where_clause="date is '2025-12-29 00:00:00'"
# )

# Store Company Data

## Company Prices

In [19]:
import utils.sql_utils as sql_utils

hist_prices = sql_utils.read_sql_table(
    database_name="CODE_CAPITAL", table_name="prices"
)

In [20]:
hist_prices

,DATE,OPEN,HIGH,LOW,CLOSE,ADJ_CLOSE,VOLUME,DIVIDENDS,STOCK_SPLITS,TICKER
0,2000-01-03,6.364583,6.385417,6.166667,6.354167,6.354167,4728000.0,0.0,0.0,FI
1,2000-01-04,6.229167,6.375000,5.833333,5.916667,5.916667,5572200.0,0.0,0.0,FI
2,2000-01-05,5.916667,6.395833,5.895833,6.328125,6.328125,5250600.0,0.0,0.0,FI
3,2000-01-06,6.250000,6.322917,6.020833,6.130208,6.130208,4111800.0,0.0,0.0,FI
4,2000-01-07,6.104167,6.395833,6.020833,6.359375,6.359375,3436200.0,0.0,0.0,FI
...,...,...,...,...,...,...,...,...,...,...
11478735,2025-12-26,53.020000,53.029999,51.700001,52.400002,52.400002,13099400.0,0.0,0.0,NVO
11478736,2025-12-24,1059.819946,1066.800049,1057.439941,1065.520020,1065.520020,229800.0,0.0,0.0,ASML
11478737,2025-12-26,1066.250000,1076.089966,1063.079956,1072.750000,1072.750000,363800.0,0.0,0.0,ASML
11478738,2025-12-24,148.029999,149.419998,148.029999,149.350006,149.350006,73700.0,0.0,0.0,LVMUY


In [5]:
all_prices = pull_prices(tickers=all_indices["TICKER"].unique())

# sql_utils.write_sql_table(
#     table_name="prices", database_name="CODE_CAPITAL", df=all_prices, overwrite=False
# )

OperationalError: (pyodbc.OperationalError) ('08S01', '[08S01] [Microsoft][ODBC Driver 18 for SQL Server]TCP Provider: Error code 0x274C (10060) (SQLExecDirectW)')
[SQL: 
    SELECT TICKER, MAX(DATE) AS START_DATE
    FROM prices
    WHERE TICKER IN ('A', 'AAPL', 'ABBV', 'ABNB', 'ABT', 'ACGL', 'ACN', 'ADBE', 'ADI', 'ADM', 'ADP', 'ADSK', 'AEE', 'AEP', 'AES', 'AFL', 'AIG', 'AIZ', 'AJG', 'AKAM', 'ALB', 'ALGN', 'ALL', 'ALLE', 'AMAT', 'AMCR', 'AMD', 'AME', 'AMGN', 'AMP', 'AMT', 'AMZN', 'ANET', 'AON', 'AOS', 'APA', 'APD', 'APH', 'APO', 'APP', 'APTV', 'ARE', 'ARES', 'ATO', 'AVB', 'AVGO', 'AVY', 'AWK', 'AXON', 'AXP', 'AZO', 'BA', 'BAC', 'BALL', 'BAX', 'BBY', 'BDX', 'BEN', 'BF-B', 'BG', 'BIIB', 'BK', 'BKNG', 'BKR', 'BLDR', 'BLK', 'BMY', 'BR', 'BRK-B', 'BRO', 'BSX', 'BX', 'BXP', 'C', 'CAG', 'CAH', 'CARR', 'CAT', 'CB', 'CBRE', 'CCI', 'CCL', 'CDNS', 'CDW', 'CEG', 'CF', 'CFG', 'CHD', 'CHRW', 'CHTR', 'CI', 'CINF', 'CL', 'CLX', 'CMCSA', 'CME', 'CMG', 'CMI', 'CMS', 'CNC', 'CNP', 'COF', 'COIN', 'COO', 

In [ ]:
adjusted = list(
    all_prices[(all_prices["DIVIDENDS"] > 0) | (all_prices["STOCK_SPLITS"] > 0)][
        "TICKER"
    ].unique()
)
print(adjusted)

if adjusted:
    adj_secs = "' ,'".join(adjusted)
    sql_utils.delete_sql_rows(
        database_name="CODE_CAPITAL",
        table_name="prices",
        where_clause=f"ticker in ('{adj_secs}')",
    )
    adjusted_prices = pull_prices(tickers=adjusted)

    sql_utils.write_sql_table(
        table_name="prices",
        database_name="CODE_CAPITAL",
        df=adjusted_prices,
        overwrite=False,
    )

2025-12-27 11:52:59 - INFO - Loading Prices


In [6]:
prices.get_prices(tickers=["AAPL", "MSFT"]).sort_index(ascending=False)

TICKER,AAPL,MSFT
DATE,,
2025-12-26,273.399994,487.709991
2025-12-24,273.809998,488.019989
2025-12-23,272.359985,486.850006
2025-12-22,270.970001,484.920013
2025-12-19,273.670013,485.920013
...,...,...
2000-01-07,0.746027,34.036129
2000-01-06,0.712287,33.597080
2000-01-05,0.779767,34.761539


## Financials

In [ ]:
run = False

today = date.today()
if today.weekday() == 0:
    all_financial_annual = pull_financials(
        tickers=all_indices["TICKER"].unique(), annual=True, statement_type="financial"
    )
    time.sleep(60 * 5)
    all_financial_quarterly = pull_financials(
        tickers=all_indices["TICKER"].unique(), annual=False, statement_type="financial"
    )
    time.sleep(60 * 5)
    all_balancesheet_annual = pull_financials(
        tickers=all_indices["TICKER"].unique(),
        annual=True,
        statement_type="balance_sheet",
    )
    time.sleep(60 * 5)
    all_balancesheet_quarterly = pull_financials(
        tickers=all_indices["TICKER"].unique(),
        annual=False,
        statement_type="balance_sheet",
    )
    time.sleep(60 * 5)
    all_income_annual = pull_financials(
        tickers=all_indices["TICKER"].unique(),
        annual=True,
        statement_type="income_statement",
    )
    time.sleep(60 * 5)
    all_income_quarterly = pull_financials(
        tickers=all_indices["TICKER"].unique(),
        annual=False,
        statement_type="income_statement",
    )
    time.sleep(60 * 5)
    all_cashflow_annual = pull_financials(
        tickers=all_indices["TICKER"].unique(), annual=True, statement_type="cashflow"
    )
    time.sleep(60 * 5)
    all_cashflow_quarterly = pull_financials(
        tickers=all_indices["TICKER"].unique(), annual=False, statement_type="cashflow"
    )
    run = True

else:
    missing = find_missing_tickers("financial_annual", all_indices["TICKER"].unique())
    print(missing)
    if missing:
        all_financial_annual = pull_financials(
            tickers=missing, annual=True, statement_type="financial"
        )
        time.sleep(60 * 5)
        all_financial_quarterly = pull_financials(
            tickers=missing, annual=False, statement_type="financial"
        )
        time.sleep(60 * 5)
        all_balancesheet_annual = pull_financials(
            tickers=missing, annual=True, statement_type="balance_sheet"
        )
        time.sleep(60 * 5)
        all_balancesheet_quarterly = pull_financials(
            tickers=missing, annual=False, statement_type="balance_sheet"
        )
        time.sleep(60 * 5)
        all_income_annual = pull_financials(
            tickers=missing, annual=True, statement_type="income_statement"
        )
        time.sleep(60 * 5)
        all_income_quarterly = pull_financials(
            tickers=missing, annual=False, statement_type="income_statement"
        )
        time.sleep(60 * 5)
        all_cashflow_annual = pull_financials(
            tickers=missing, annual=True, statement_type="cashflow"
        )
        time.sleep(60 * 5)
        all_cashflow_quarterly = pull_financials(
            tickers=missing, annual=False, statement_type="cashflow"
        )
        run = True

2025-12-30 10:33:56 - INFO - Loading Company Financial
2025-12-30 10:34:17 - INFO - Loading Company Financial
2025-12-30 10:34:39 - INFO - Loading Company Balance Sheet
2025-12-30 10:35:00 - INFO - Loading Company Balance Sheet
2025-12-30 10:35:09 - INFO - Loading Company Income Statement
2025-12-30 10:35:17 - INFO - Loading Company Income Statement
2025-12-30 10:35:25 - INFO - Loading Company Cashflow
2025-12-30 10:35:33 - INFO - Loading Company Cashflow


In [7]:
if run:
    azure_utils.write_sql_table(
        table_name="financial_annual",
        engine=engine,
        df=all_financial_annual,
        overwrite=False,
    )
    azure_utils.write_sql_table(
        table_name="financial_quarterly",
        engine=engine,
        df=all_financial_quarterly,
        overwrite=False,
    )
    azure_utils.write_sql_table(
        table_name="balancesheet_annual",
        engine=engine,
        df=all_balancesheet_annual,
        overwrite=False,
    )
    azure_utils.write_sql_table(
        table_name="balancesheet_quarterly",
        engine=engine,
        df=all_balancesheet_quarterly,
        overwrite=False,
    )
    azure_utils.write_sql_table(
        table_name="incomestatement_annual",
        engine=engine,
        df=all_income_annual,
        overwrite=False,
    )
    azure_utils.write_sql_table(
        table_name="incomestatement_quarterly",
        engine=engine,
        df=all_income_quarterly,
        overwrite=False,
    )
    azure_utils.write_sql_table(
        table_name="cashflow_annual",
        engine=engine,
        df=all_cashflow_annual,
        overwrite=False,
    )
    azure_utils.write_sql_table(
        table_name="cashflow_quarterly",
        engine=engine,
        df=all_cashflow_quarterly,
        overwrite=False,
    )

## Company Info

In [20]:
run = False

today = date.today()
if today.weekday() == 0:
    client = create_client(tickers=all_indices["TICKER"].unique())
    all_info = pull_info(tickers=all_indices["TICKER"].unique(), client=client)
    all_officers = pull_officers(tickers=all_indices["TICKER"].unique(), client=client)
    run = True
else:
    missing = find_missing_tickers("company_info", all_indices["TICKER"].unique())
    print(missing)
    if missing:
        client = create_client(tickers=missing)
        all_info = pull_info(tickers=missing, client=client)
        all_officers = pull_officers(tickers=missing, client=client)
        run = True

[]


In [8]:
if run:
    azure_utils.write_sql_table(
        table_name="company_info",
        engine=engine,
        df=all_info,
        overwrite=False,
    )
    azure_utils.write_sql_table(
        table_name="officers",
        engine=engine,
        df=all_officers,
        overwrite=False,
    )

In [14]:
info_df = company_info.get_company_info(tickers=None)
cachedf = azure_utils.read_sql_table(engine=engine, table_name="address")

missing_address = info_df[~(info_df["ADDRESS1"].isin(cachedf["ADDRESS1"].unique()))]
missing_address = missing_address.dropna(subset="ADDRESS1")

missing_address

,DATE,TICKER,ADDRESS1,CITY,STATE,ZIP,COUNTRY,PHONE,WEBSITE,INDUSTRY,...,REGULARMARKETDAYRANGE,REGULARMARKETCHANGEPERCENT,REGULARMARKETPRICE,DISPLAYNAME,TRAILINGPEGRATIO,IPOEXPECTEDDATE,ADDRESS2,FAX,INDUSTRYSYMBOL,PEGRATIO


In [15]:
from utils import geo

df = geo.geocode_dataframe(missing_address, cache_df=cachedf, delay=0.5)

df = df[["ADDRESS1", "CITY", "COUNTRY", "LAT", "LON"]].dropna(subset="LAT")

df

Geocoding: 0it [00:00, ?it/s]


,ADDRESS1,CITY,COUNTRY,LAT,LON


In [16]:
azure_utils.write_sql_table(engine=engine, table_name="address", df=df, overwrite=False)